In [1]:
# http://peterroelants.github.io/posts/rnn_implementation_part01/

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import LogNorm
%matplotlib inline
np.random.seed(seed=1)

In [2]:
# データセットを作成
# 01からなる長さ10の系列が20サンプル
nb_of_samples = 20
sequence_len = 10

X = np.zeros((nb_of_samples, sequence_len))
for row_idx in range(nb_of_samples):
    X[row_idx, :] = np.around(np.random.rand(sequence_len))
X = X.astype(int)
print(X.shape)

# ターゲットは各サンプルの1の数（回帰問題）
t = np.sum(X, axis=1)
print(t.shape)

(20, 10)
(20,)


In [3]:
# forward処理の関数
def update_state(xk, sk, wx, wRec):
    return sk * wRec + xk * wx

def forward_states(X, wx, wRec):
    # すべての時系列サンプルの状態を記憶する行列 S[サンプル, 状態]
    # unfoldすると時系列長の分だけ状態がある
    # S[0]は初期状態とする（0で初期化）
    # S[:, -1]が各時系列を入力したときの最終出力 = y
    S = np.zeros((X.shape[0], X.shape[1] + 1))
    for k in range(0, X.shape[1]):
        S[:, k + 1] = update_state(X[:, k], S[:, k], wx, wRec)
    
    return S

def cost(y, t):
    return ((t - y) ** 2).sum() / nb_of_samples

In [5]:
def output_gradient(y, t):
    return 2.0 * (y - t) / nb_of_samples

def backward_gradient(X, S, grad_out, wRec):
    grad_over_time = np.zeros((X.shape[0], X.shape[1] + 1))
    grad_over_time[:, -1] = grad_out
    
    # 各レイヤでパラメータを共有しているため勾配を足し合わせる
    wx_grad = 0
    wRec_grad = 0
    for k in range(X.shape[1], 0, -1):
        wx_grad += np.sum(grad_over_time[:, k] * X[:, k-1])
        wRec_grad += np.sum(grad_over_time[:, k] * S[:, k-1])
        grad_over_time[:, k-1] = grad_over_time[:, k] * wRec
    return (wx_grad, wRec_grad), grad_over_time

In [6]:
# gradient checking
params = [1.2, 1.2]  # [wx, wRec]
eps = 1e-7
S = forward_states(X, params[0], params[1])
grad_out = output_gradient(S[:, -1], t)
backprop_grads, grad_over_time = backward_gradient(X, S, grad_out, params[1])
print(backprop_grads)
for p_idx, _ in enumerate(params):
    grad_backprop = backprop_grads[p_idx]
    params[p_idx] += eps
    plus_cost = cost(forward_states(X, params[0], params[1])[:, -1], t)
    params[p_idx] -= 2 * eps
    min_cost = cost(forward_states(X, params[0], params[1])[:, -1], t)
    params[p_idx] += eps
    grad_num = (plus_cost - min_cost) / (2 * eps)
    if not np.isclose(grad_num, grad_backprop):
        raise ValueError('Numerical gradient of {:.6f} is not close to the backpropagation gradient of {:.6f}!'.format(float(grad_num), float(grad_backprop)))  
print('No gradient errors found')

(274.59542913321798, 1593.2428444791924)
No gradient errors found


In [13]:
def update_rprop(X, t, W, W_prev_sign, W_delta, eta_p, eta_n):
    # 各エポックで呼ばれる
    # forward_states()の中で状態Sは毎回初期化している
    S = forward_states(X, W[0], W[1])
    grad_out = output_gradient(S[:, -1], t)
    W_grads, _ = backward_gradient(X, S, grad_out, W[1])
    W_sign = np.sign(W_grads)
    for i, _ in enumerate(W):
        if W_sign[i] == W_prev_sign[i]:
            W_delta[i] *= eta_p
        else:
            W_delta[i] *= eta_n
    return W_delta, W_sign

In [14]:
eta_p = 1.2
eta_n = 0.5

W = [-1.5, 2]  # [wx, wRec]
W_delta = [0.001, 0.001]
W_sign = [0, 0]

ls_of_ws = [(W[0], W[1])]
for i in range(500):
    W_delta, W_sign = update_rprop(X, t, W, W_sign, W_delta, eta_p, eta_n)
    for i, _ in enumerate(W):
        W[i] -= W_sign[i] * W_delta[i]
    ls_of_ws.append((W[0], W[1]))

print('Final weights are: wx = {0}, wRec = {1}'.format(W[0], W[1]))

S: [[    0.      0.     -1.5    -3.     -6.    -12.    -24.    -48.    -96.
   -192.   -385.5]
 [    0.      0.     -1.5    -3.     -7.5   -15.    -31.5   -63.   -127.5
   -255.   -510. ]
 [    0.     -1.5    -4.5    -9.    -19.5   -40.5   -82.5  -165.   -330.
   -660.  -1321.5]
 [    0.      0.      0.     -1.5    -4.5   -10.5   -21.    -43.5   -88.5
   -177.   -355.5]
 [    0.     -1.5    -4.5    -9.    -19.5   -39.    -78.   -157.5  -315.
   -630.  -1260. ]
 [    0.      0.     -1.5    -3.     -6.    -12.    -24.    -49.5   -99.
   -199.5  -400.5]
 [    0.      0.      0.     -1.5    -3.     -6.    -13.5   -28.5   -58.5
   -118.5  -238.5]
 [    0.     -1.5    -3.     -6.    -13.5   -27.    -54.   -109.5  -219.
   -439.5  -880.5]
 [    0.     -1.5    -4.5   -10.5   -21.    -42.    -85.5  -171.   -343.5
   -688.5 -1378.5]
 [    0.      0.     -1.5    -3.     -7.5   -15.    -30.    -61.5  -124.5
   -249.   -499.5]
 [    0.      0.     -1.5    -4.5    -9.    -19.5   -40.5   -81.   -163.

In [9]:
# 入力は任意の長さでOK
test_input = np.asmatrix([[0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ,1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ,1]])
test_output = forward_states(test_input, W[0], W[1])

In [10]:
test_input.sum()

29

In [11]:
test_output.shape

(1, 37)

In [12]:
test_output[:, -1][0]

28.902835057275531